In [1]:
!pip install gdown

In [2]:
pip install keras imbalanced-learn

In [3]:
!gdown --id '1LgI2Kh4XfwCf_9FtHRS51nmdMx0sfvhd'

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1LgI2Kh4XfwCf_9FtHRS51nmdMx0sfvhd
To: /content/review.xlsx
100% 2.50M/2.50M [00:00<00:00, 131MB/s]


In [4]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'


In [5]:
import pandas as pd

In [6]:
df = pd.read_excel('review.xlsx')
df

,title,text,aspek,opini,sentimen
0,Katenjo Chill Space & kopi Bar,Tempatnya lumayan luas,tempat,luas,positif
1,Katenjo Chill Space & kopi Bar,Lokasi strategis,lokasi,strategis,positif
2,Katenjo Chill Space & kopi Bar,Makanan lumayan enak,makanan,enak,positif
3,Katenjo Chill Space & kopi Bar,Harga cukup terjangkau,harga,terjangkau,positif
4,Katenjo Chill Space & kopi Bar,Pelayan bertanggung jawab,pelayan,bertanggung jawab,positif
...,...,...,...,...,...
53463,Rumah Kopi Baretto,Rahal,NaN,NaN,NaN
53464,Rumah Kopi Baretto,bagus,NaN,NaN,NaN
53465,Rumah Kopi Baretto,Bagus sekali,NaN,NaN,NaN
53466,Rumah Kopi Baretto,berlari,NaN,NaN,NaN


In [7]:
df_new = df.dropna(subset=['opini'])
df_new

,title,text,aspek,opini,sentimen
0,Katenjo Chill Space & kopi Bar,Tempatnya lumayan luas,tempat,luas,positif
1,Katenjo Chill Space & kopi Bar,Lokasi strategis,lokasi,strategis,positif
2,Katenjo Chill Space & kopi Bar,Makanan lumayan enak,makanan,enak,positif
3,Katenjo Chill Space & kopi Bar,Harga cukup terjangkau,harga,terjangkau,positif
4,Katenjo Chill Space & kopi Bar,Pelayan bertanggung jawab,pelayan,bertanggung jawab,positif
...,...,...,...,...,...
1332,Toga Peak Cafe,Caffe dengan pemandangan pemandangan kota sume...,pemandangan,cityskype,positif
1333,Toga Peak Cafe,buat yang berkunjung di sumedang silahkan data...,suasana,ciamik,positif
1334,Toga Peak Cafe,"Tempatnya enak, sambil nongkrong, makan, ngemi...",tempat,enak,positif
1335,Toga Peak Cafe,"Pelayan nya ga ramah, ga bisa nambah kursi, pa...",pelayanan,ga ramah,negatif


In [8]:
df_new = df_new.applymap(lambda s:s.lower() if type(s) == str else s)
df_new['sentimen'] = df_new['sentimen'].replace("posiif", "positif")
df_new.dropna(axis=0, inplace=True)

In [9]:
df_new["sentimen"].value_counts()

positif    942
negatif    117
Name: sentimen, dtype: int64

In [10]:
df_new.isnull().sum()

title       0
text        0
aspek       0
opini       0
sentimen    0
dtype: int64

In [11]:
df_new

,title,text,aspek,opini,sentimen
0,katenjo chill space & kopi bar,tempatnya lumayan luas,tempat,luas,positif
1,katenjo chill space & kopi bar,lokasi strategis,lokasi,strategis,positif
2,katenjo chill space & kopi bar,makanan lumayan enak,makanan,enak,positif
3,katenjo chill space & kopi bar,harga cukup terjangkau,harga,terjangkau,positif
4,katenjo chill space & kopi bar,pelayan bertanggung jawab,pelayan,bertanggung jawab,positif
...,...,...,...,...,...
1332,toga peak cafe,caffe dengan pemandangan pemandangan kota sume...,pemandangan,cityskype,positif
1333,toga peak cafe,buat yang berkunjung di sumedang silahkan data...,suasana,ciamik,positif
1334,toga peak cafe,"tempatnya enak, sambil nongkrong, makan, ngemi...",tempat,enak,positif
1335,toga peak cafe,"pelayan nya ga ramah, ga bisa nambah kursi, pa...",pelayanan,ga ramah,negatif


In [13]:
import pandas as pd
import string
import re
import json
def preprocess_text(text):
    # Path to your slang dictionary
    slang_path = '/content/_json_colloquial-indonesian-lexicon.txt'

    # Load your slang dictionary
    with open(slang_path, 'r') as f:
        slang_dict = json.load(f)

    # Convert the text to string
    text = str(text)

    # Convert the text to lowercase
    text = text.lower()

    # Tokenize the text
    text = text.split()

    # Replace slang words and remove stopwords
    text = [slang_dict.get(word, word) for word in text]

        # Join the words back into a single string
    text = ' '.join(text)

    return text


# Apply the function to the DataFrame in parallel
df_new['text'] = df_new['text'].apply(preprocess_text)


In [14]:
df_new

,title,text,aspek,opini,sentimen
0,katenjo chill space & kopi bar,tempatnya lumayan luas,tempat,luas,positif
1,katenjo chill space & kopi bar,lokasi strategis,lokasi,strategis,positif
2,katenjo chill space & kopi bar,makanan lumayan enak,makanan,enak,positif
3,katenjo chill space & kopi bar,harga cukup terjangkau,harga,terjangkau,positif
4,katenjo chill space & kopi bar,pelayan bertanggung jawab,pelayan,bertanggung jawab,positif
...,...,...,...,...,...
1332,toga peak cafe,caffe dengan pemandangan pemandangan kota sume...,pemandangan,cityskype,positif
1333,toga peak cafe,buat yang berkunjung di sumedang silakan datan...,suasana,ciamik,positif
1334,toga peak cafe,"tempatnya enak, sambil nongkrong, makan, ngemi...",tempat,enak,positif
1335,toga peak cafe,"pelayan nya enggak ramah, enggak bisa menambah...",pelayanan,ga ramah,negatif


In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.optimizers import Adam
import numpy as np

X = df_new['text'] #feature
y = df_new['sentimen'] #label

# Load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1')
model = TFBertForSequenceClassification.from_pretrained('indobenchmark/indobert-base-p1', num_labels=y.nunique(), hidden_dropout_prob=0.5, attention_probs_dropout_prob=0.5)

# change X to list string
X_list = X.tolist()

# Tokenize the input data
encoded_inputs = tokenizer.batch_encode_plus(X_list, truncation=True, padding=True, max_length=128)

# Convert the input data to tensors
input_ids = np.array(encoded_inputs["input_ids"])
attention_mask = np.array(encoded_inputs["attention_mask"])

# Encode the labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Compile the model
optimizer = Adam(learning_rate=5e-5)
loss = 'sparse_categorical_crossentropy'
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

BATCH_SIZE = 32
EPOCHS = 10
# use input_ids and attention_mask as input to the model
history = model.fit([input_ids, attention_mask], y_encoded, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=0.1)


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
24/30 [=======================>......] - ETA: 4:52 - loss: 0.8467 - accuracy: 0.8763

In [ ]:
from transformers import BertTokenizer, TFBertForTokenClassification
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Assume df_new is your DataFrame and 'text' is the feature and 'aspek', 'opini', 'sentimen' are the labels
X = df_new['text']
y = df_new[['aspek', 'opini', 'sentimen']].values.tolist()

# Load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1')
model = TFBertForTokenClassification.from_pretrained('indobenchmark/indobert-base-p1', num_labels=len(set([label for sublist in y for label in sublist])))

# Convert X into a list of strings
X_list = X.tolist()

# Tokenize the input data and convert to input tensors
encoded_inputs = tokenizer(X_list, truncation=True, padding=True, max_length=128, is_split_into_words=True)

# Convert the input data to tensors
input_ids = np.array(encoded_inputs["input_ids"])
attention_mask = np.array(encoded_inputs["attention_mask"])

# Flatten your list of labels and binarize them
lb = LabelEncoder()
y_flattened = [label for sublist in y for label in sublist]
y_encoded = lb.fit_transform(y_flattened)

# Reshape y_encoded to have the same 2D shape as input_ids
y_encoded = y_encoded.reshape(input_ids.shape)

# Compile the model
optimizer = Adam(learning_rate=5e-5)
loss = 'sparse_categorical_crossentropy'
model.compile(optimizer=optimizer, loss=loss)

BATCH_SIZE = 32  # You can adjust this value according to your needs
EPOCHS = 10  # You can adjust this value according to your needs

# Now you can use input_ids and attention_mask as input to your model
history = model.fit([input_ids, attention_mask], y_encoded, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=0.1)


All model checkpoint layers were used when initializing TFBertForTokenClassification.

Some layers of TFBertForTokenClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: ignored

# **TIDAK DIPAKAI**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import LSTM, Embedding, Dense, Input
from keras.layers import TimeDistributed, SpatialDropout1D, Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

# asumsikan df_new adalah DataFrame Anda dan 'text' adalah fitur dan 'sentimen' adalah label
X = df_new['text']
y = df_new['sentimen']

# Ubah teks menjadi fitur numerik menggunakan TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

# Hitung jumlah kata unik dalam data Anda
n_words = len(vectorizer.get_feature_names_out())

# Hitung jumlah label unik dalam data Anda
n_tags = y.nunique()

# Bagi data menjadi set pelatihan dan pengujian
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Lakukan oversampling pada kelas minoritas
ros = RandomOverSampler(random_state=0)
X_resampled_over, y_resampled_over = ros.fit_resample(X_train.toarray(), y_train)

# Lakukan undersampling pada kelas mayoritas
rus = RandomUnderSampler(random_state=0)
X_resampled_under, y_resampled_under = rus.fit_resample(X_train.toarray(), y_train)

# Tentukan panjang maksimum dari sekuen dalam data Anda
MAX_LEN = X_train.shape[1]  # Use the number of features as the sequence length

# Tentukan dimensi embedding
EMBEDDING = 100

# Membuat model
input = Input(shape=(MAX_LEN,))
model = Embedding(input_dim=n_words + 2, output_dim=EMBEDDING, input_length=MAX_LEN)(input)
model = SpatialDropout1D(0.1)(model)
model = Bidirectional(LSTM(units=50, return_sequences=True, recurrent_dropout=0.1))(model)
out = TimeDistributed(Dense(n_tags + 1, activation="softmax"))(model)

model = Model(input, out)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

BATCH_SIZE = 32  # You can adjust this value according to your needs
EPOCHS = 10  # You can adjust this value according to your needs

# Ubah teks menjadi urutan token
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_seq = tokenizer.texts_to_sequences(X)

# Pad sequences to the same length
X_pad = pad_sequences(X_seq, maxlen=MAX_LEN)

# Now you can use X_pad as input to your model
history = model.fit(X_pad, np.array(y_train), batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=0.1, verbose=2)

# history = model.fit(X_train.toarray(), np.array(y_train), batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=0.1, verbose=1)


AttributeError: ignored

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split

# asumsikan X adalah fitur dan y adalah label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

rus = RandomUnderSampler(random_state=0)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)
